In [95]:
import json
import spacy
from spacy.language import Language
import dotenv
dotenv.load_dotenv('.env')

True

In [23]:
file_in=''

In [352]:
with open(r'C:\Users\navcer\Downloads\biz\json\20220304652-Invoice_2071790_from_Intone_Networks_Inc-0.json','r') as file:
    file_in=json.loads(file.read())



In [353]:
ext_str=""

In [354]:
for i in range(len(file_in['text_annotations'])):
    ext_str+=file_in['text_annotations'][i]['block_details']['block_description']+'\n' 
    

In [355]:
ext_str

"Intone Networks Inc\n10 Austin Ave\nIselin, NJ 08830\nintone\n9089307189\nN E T W o R K S\naccounts@intonenetworks.con\nhttp://intonenetworks.com\nINVOICE\nBILL TO\nINVOICE # 2071790\nBiztech2go Inc\nDATE 09/04/2020\n2082 Bus Ctr Dr Ste 250\nDUE DATE 10/04/2020\nIrvine, CA 92612\nTERMS Net 30\nUnited States\nSTATE\nSTART DATE\nEND DATE\nCA\n08/01/2020\n08/31/2020\nACTIVITY\nQTY\nRATE\nAMOUNT\nBala Gade Manoj Kumar / Biztech2go\n168\n85.00\n14,280.00\nServices provided by our consultant Gade Bala Manoj Kumar for\nthe month of August'2020\nBALANCE DUE\n$14,280.00\n"

In [367]:
from spacy.matcher import Matcher

In [381]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# doc=nlp(ext_str)

In [395]:
type(nlp)

spacy.lang.en.English

In [382]:
invoice_pat=[[{'text':{'in':['INVOICE','BILL']}},{'ORTH':{'IN':['#','@']}},{'IS_DIGIT':True}]]
# pattern to extract balance/money statement
balance_pat=[[{'IS_CURRENCY': True}, {'IS_SPACE': True, 'OP': '?'}, {'LIKE_NUM': True}]]
# pattern to extract date eg DD-MM-YYY DD/MM/YYYY etc
date_pat=[[{'text':{'REGEX':r'\b(\d{4}|\d{2})[\/\-]\d{2}[\/\-](\d{4}|\d{2})'}}],
[{'IS_DIGIT': True,'length':{'in':[2,4]}}, {'ORTH': '-'}, {'IS_DIGIT': True,'length':2}, {'ORTH': '-'}, {'IS_DIGIT': True,'length':{'in':[2,4]}}],
[{'IS_DIGIT': True}, {'ORTH': '-'}, {'IS_ALPHA': True}, {'ORTH': '-'}, {'IS_DIGIT': True}],
[{'IS_ALPHA': True}, {'ORTH': '-'}, {'IS_DIGIT': True}, {'ORTH': '-'}, {'IS_DIGIT': True}]
]

# add matcher pattern for balance,date and invoice
matcher.add('bal_matcher',balance_pat)
matcher.add('dat_matcher',date_pat)
matcher.add('inv_matcher',invoice_pat)
# custom pipeline component to add custom NER to doc.ent 
@Language.component("custom_ent")
def custom_ent(doc):
    new_ents=[]
    matches=matcher(doc)
    saved_ents=set()
    # check for required matches
    for i in matches:
        string_id=nlp.vocab.strings[i[0]]
        # if first of its type match date, balance
        if string_id not in saved_ents:
            if string_id=='bal_matcher':
                new_ent=('balance',i[1],i[2],)
                saved_ents.add(string_id)
                new_ents.append(new_ent)
                continue
            if string_id=='dat_matcher' :
                new_ent=('due_date',i[1],i[2],)
                saved_ents.add(string_id)
                new_ents.append(new_ent)
                continue
            if string_id=='inv_matcher' :
                new_ent=('invoice_number',i[1]+2,i[2],)
                saved_ents.add(string_id)
                new_ents.append(new_ent)
                continue
    # check for organization ent from the default statstical model of spacy
    for ents in doc.ents:
        if ents.label_ and (ents.label_ not in saved_ents):
            new_ent=('vendor_name',ents.start,ents.end)
            new_ents.append(new_ent)
            break
        # override the default ents
    doc.ents=new_ents
    return doc
    


In [383]:
# add new component to nlp pipeline after NER
nlp.add_pipe('custom_ent',after='ner')

<function __main__.custom_ent(doc)>

In [384]:
# run pipe over the exracted string
doc=nlp(ext_str)
# print(doc)
for i in doc.ents:
    print(i)
#     # pass

Intone Networks Inc
2071790
09/04/2020
$14,280.00
